# Training a model in TF

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.contrib.learn.python.learn import datasets
dataset = datasets.load_iris()

# split into input data (x) and GT labels (y)
data_x = dataset.data
data_y = dataset.target
print data_x.shape
print data_y.shape

(150, 4)
(150,)


# Step 1: Building the computation graph

In [3]:
# placeholder for data x (784-dim)
x    = tf.placeholder("float", shape=[None, 4])
y_GT = tf.placeholder("int64", shape=[None, ])

# model parameters
n_hidden = 20
W_h = tf.Variable(0.1*tf.random_normal([4, n_hidden]), name="W_h")
b_h = tf.Variable(tf.random_normal([n_hidden]), name="b_h")
hidden_layer = tf.matmul(x, W_h) + b_h

# model parameters
W = tf.Variable(0.1*tf.random_normal([n_hidden, 3]), name="W")
b = tf.Variable(tf.zeros([3]), name="b")

# putting the model together
y = tf.nn.softmax(tf.matmul(hidden_layer, W) + b)

In [4]:
print y

Tensor("Softmax:0", shape=(?, 3), dtype=float32)


# Step 2: Specify loss and training/optimization

In [5]:
# cross-entropy loss
#cross_entropy = -tf.reduce_sum(tf.one_hot(y_GT, depth=3) * tf.log(y+1e-10))
cross_entropy = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y_GT, depth=3), logits=(tf.matmul(hidden_layer, W) + b)))
# define optimizer
opt = tf.train.GradientDescentOptimizer(0.0001)
train_op = opt.minimize(cross_entropy)

In [6]:
cross_entropy

<tf.Tensor 'Sum:0' shape=() dtype=float32>

# Step 3: Launch a session

In [7]:
# create operation to inialize all variables
init_op = tf.initialize_all_variables()

In [8]:
# launch session
with tf.Session() as sess:
    sess.run(init_op)
    
    print W
    print sess.run(W) #print W.eval()

[[ 0.09495648 -0.11181545 -0.06443463]
 [ 0.05603005  0.03342399 -0.12521382]
 [ 0.0764797  -0.01175073 -0.12406522]
 [-0.00504424 -0.08993544  0.11662994]
 [-0.10030112  0.01447914 -0.06445155]
 [-0.02810389 -0.03619912  0.01265281]
 [-0.10191865 -0.05726812  0.05041992]
 [-0.19695517  0.28932351 -0.03475902]
 [-0.17699818  0.23104993 -0.06288   ]
 [ 0.06703613 -0.02751589  0.08142088]
 [-0.2342333  -0.08095408 -0.00722648]
 [-0.02683609  0.03968557 -0.14740972]
 [ 0.00886339 -0.00192394 -0.18078734]
 [ 0.04577922 -0.10625563  0.18171819]
 [ 0.04818371  0.07144029  0.14268406]
 [-0.02679454  0.15762183 -0.12937291]
 [ 0.20441477  0.15740044 -0.09919591]
 [ 0.22699249 -0.07965782  0.00641393]
 [-0.02305623  0.03198144 -0.10445283]
 [ 0.01721368  0.01479873 -0.22101191]]


In [9]:
# one step of training
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(train_op, feed_dict={x: data_x, y_GT: data_y})
    
    # something should have happened
    #print sess.run(W), sess.run(b)
    #print sess.run(y, feed_dict={x:data_x, y_GT:data_y})
    
    xen = sess.run(cross_entropy, feed_dict={x: data_x, y_GT: data_y})
    print xen
    

239.32


In [10]:
# summary 
tf.scalar_summary("cross_entropy", cross_entropy)
summary_op = tf.merge_all_summaries()

In [11]:
# multiple steps of training
accuracy = tf.reduce_mean(tf.cast(tf.equal(y_GT, tf.argmax(y, 1)), tf.float32))
    
with tf.Session() as sess:
    sess.run(init_op)
    
    summary_writer  = tf.train.SummaryWriter('tf_logs', sess.graph)
    for iterIndex in range(100):
        sess.run(train_op, feed_dict={x: data_x, y_GT: data_y})
    
        xen = sess.run(cross_entropy, feed_dict={x: data_x, y_GT: data_y})
        print("Iter %d -- Cross-entropy: %f"%(iterIndex,xen))
        
        summary = sess.run(summary_op, feed_dict={x: data_x, y_GT: data_y})
        summary_writer.add_summary(summary, iterIndex)
        
        if iterIndex%10==0:
            acc = sess.run(accuracy, feed_dict={x: data_x, y_GT: data_y})
            print acc
        


Iter 0 -- Cross-entropy: 283.711090
0.333333
Iter 1 -- Cross-entropy: 327.863190
Iter 2 -- Cross-entropy: 145.388794
Iter 3 -- Cross-entropy: 131.460129
Iter 4 -- Cross-entropy: 123.338280
Iter 5 -- Cross-entropy: 115.632996
Iter 6 -- Cross-entropy: 108.497047
Iter 7 -- Cross-entropy: 102.022621
Iter 8 -- Cross-entropy: 96.261314
Iter 9 -- Cross-entropy: 91.182167
Iter 10 -- Cross-entropy: 86.723236
0.826667
Iter 11 -- Cross-entropy: 82.802017
Iter 12 -- Cross-entropy: 79.337799
Iter 13 -- Cross-entropy: 76.255409
Iter 14 -- Cross-entropy: 73.490891
Iter 15 -- Cross-entropy: 70.990128
Iter 16 -- Cross-entropy: 68.709152
Iter 17 -- Cross-entropy: 66.611923
Iter 18 -- Cross-entropy: 64.669525
Iter 19 -- Cross-entropy: 62.858398
Iter 20 -- Cross-entropy: 61.159687
0.966667
Iter 21 -- Cross-entropy: 59.557945
Iter 22 -- Cross-entropy: 58.040806
Iter 23 -- Cross-entropy: 56.598026
Iter 24 -- Cross-entropy: 55.221390
Iter 25 -- Cross-entropy: 53.903976
Iter 26 -- Cross-entropy: 52.640373
Ite